In [36]:
import keras 
import os
import numpy as np
import cv2
import tensorflow as tf
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image 
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import RMSprop
from keras.models import Sequential
import keras
from keras import layers
from keras.layers import Flatten
from sklearn.model_selection import train_test_split

In [37]:
shapy=cv2.imread("C:/Users/ELCOT/Desktop/Mask/data/test/0/0.jpg")
shapy.shape

(50, 50, 3)

In [38]:
x,y=[],[]
path="C:/Users/ELCOT/Desktop/Mask/data/test"
loc=os.listdir(path)
for i in os.listdir(path):
        for j in os.listdir(path+'/'+i):
                im= Image.open(path+'/'+i+'/'+j)
                im=im.resize((64,64),Image.ANTIALIAS)
                im=np.array(im)
                
                x.append(im)
                y.append(loc.index(i))
x=np.array(x)
y=to_categorical(y,num_classes=len(loc))

In [39]:
x.shape,y.shape

((2828, 64, 64, 3), (2828, 3))

In [40]:
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.80,shuffle=True,random_state=True)
x_t,x_val,y_t,y_val=train_test_split(x_train,y_train,train_size=0.80,shuffle=True,random_state=True)

In [42]:
model=Sequential()
model.add(keras.Input(shape=(64,64,3)))
model.add(layers.Conv2D(34,5,strides=5,activation="relu"))
model.add(layers.Conv2D(64,3,strides=3,activation="relu"))
model.add(layers.MaxPooling2D(3))
model.summary()
model.add(layers.GlobalMaxPooling2D())
model.add(Flatten())
model.add(layers.Dense(512,activation="relu"))
model.add(layers.Dense(256,activation="relu"))
model.add(layers.Dense(32,activation="relu"))
model.add(layers.Dense(3, activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
history=model.fit(x_t,y_t,validation_data=(x_val,y_val),epochs = 50,batch_size =100)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 12, 12, 34)        2584      
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 4, 4, 64)          19648     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 1, 1, 64)          0         
Total params: 22,232
Trainable params: 22,232
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
19/19 [==============================] - 8s 159ms/step - loss: 4.1439 - accuracy: 0.5653 - val_loss: 0.1846 - val_accuracy: 0.9227
Epoch 2/50
19/19 [==============================] - 2s 104ms/step - loss: 0.2007 - accuracy: 0.9253 - val_loss: 0.0926 - val_accuracy: 0.9757
Epoch 3/50
19/19 [==============================] - 2s 108ms/step - loss: 0.0612 - accuracy: 0.9

In [43]:
model.save('C:/Users/ELCOT/Desktop/Mask/model.h5')

In [44]:
predict_val=model.predict(x_test)
t=np.argmax(predict_val,axis=-1)

In [45]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
confusion_matrix(t,np.argmax(y_test,axis=1))

array([[276,   7,   0],
       [  4, 124,   1],
       [  0,   0, 154]], dtype=int64)

In [46]:
accuracy_score(t,np.argmax(y_test,axis=1))*100

97.87985865724382